In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
load_dotenv()

True

In [2]:
# Equation for mean average percentage error for test the model performance
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs((y_true - y_pred))) / np.sum(np.abs(y_true))

In [3]:
# Database connection and query
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

df_productsales = pd.read_sql("SELECT * FROM core_productsale;", conn)
df_products = pd.read_sql("SELECT * FROM core_product;", conn)
df_sales = pd.read_sql("SELECT * FROM core_sale;", conn)
df_products

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn

,id,name,description,presentation,cost,price_1,price_2,price_3,brand,category_id,image,cost_currency,price_1_currency,price_2_currency,price_3_currency,code
0,1,REQUESON DURO CON SAL (E),Nan,KG,1.10,1.42,1.30,1.38,Yaracal,23,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,000005
1,2,SUERO DE LECHE MIS QUESOS 700 ML,Nan,UND,1.07,1.47,1.25,1.33,Mis quesos,8,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00001
2,3,COMPRA MATERIALES Y EQUIPOS DE OFICINA,Materiales y equipos de oficina,NAN,0.00,0.00,0.00,0.00,Nan,1056,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00003
3,4,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),Queso blaco blando extragraso,KG,4.95,6.04,5.69,5.89,El tunal,23,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00004
4,5,CREMOSITO CHEDDAR TINA 220 GRS DERILAC,Cremosito cheddar 220 grs,UND,1.50,1.90,1.74,1.84,Derilac,8,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,485,YOGURT GRIEGO 16 OZ NATUGURT,Nan,UND,3.00,3.90,3.80,3.90,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO022
485,486,YOGURT NATURAL 16 OZ NATUGURT,Nan,UND,2.00,2.80,2.70,2.80,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO023
486,487,YOGURT DE COCO NATUGURT,Nan,UND,3.00,3.90,3.80,3.90,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO024
487,488,QUESO DE YOGURT 16 OZ NATUGURT,Nan,UND,3.00,4.00,3.80,3.90,Natugurt,1063,https://i.ibb.co/SrMrfyV/pngwing-com.png,USD,USD,USD,USD,YO025


In [4]:
df_productsales

,id,quantity,income,product_id,sale_id,income_currency
0,16921,4.10,14.35,17,00034279,USD
1,16953,4.00,14.00,17,00034290,USD
2,16930,8.10,43.52,22,00034282,USD
3,17005,6.80,21.87,23,00034311,USD
4,16906,24.00,46.72,33,00034269,USD
...,...,...,...,...,...,...
71828,18772,25.30,0.00,489,00034825,USD
71829,19432,4.30,0.00,489,00035023,USD
71830,19630,8.70,0.00,489,00035079,USD
71831,59878,4.40,0.00,489,00001021,USD


In [5]:
df_merged = pd.merge(df_productsales, df_products, left_on="product_id", right_on="id", suffixes=("_productsale", "_product"))
df_merged.drop(columns=["brand", "image", "cost_currency", "price_1_currency", "price_2_currency", "price_3_currency", "code", "price_1", "price_2", "price_3", "description", "presentation", "income_currency"], inplace=True)
df_merged.set_index("id_product", inplace=True)
df_merged

,id_productsale,quantity,income,product_id,sale_id,name,cost,category_id
id_product,,,,,,,,
17,16921,4.1,14.35,17,00034279,QUESO TELITA CACHAPERO (E),2.70,23
17,16953,4.0,14.00,17,00034290,QUESO TELITA CACHAPERO (E),2.70,23
17,17217,3.6,12.60,17,00034379,QUESO TELITA CACHAPERO (E),2.70,23
17,17330,3.4,11.90,17,00034415,QUESO TELITA CACHAPERO (E),2.70,23
17,17334,3.4,11.90,17,00034418,QUESO TELITA CACHAPERO (E),2.70,23
...,...,...,...,...,...,...,...,...
6,59532,6.0,16.50,6,004428,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,2.75,54
6,81979,11.0,30.25,6,NE010339,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,2.75,54
6,82805,6.0,16.50,6,NE010715,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,2.75,54


In [6]:
df_merged_sales_with_products = pd.merge(df_merged, df_sales, left_on="sale_id", right_on="id", suffixes=("_productsale", "_sale"))
df_merged_sales_with_products["year"] = df_merged_sales_with_products.apply(lambda row: row["date"].year, axis=1)
df_merged_sales_with_products["month"] = df_merged_sales_with_products.apply(lambda row: row["date"].month, axis=1)
df_merged_sales_with_products.drop(columns=["id", "description", "income_currency", "status", "date" ], inplace=True)
df_merged_sales_with_products

,id_productsale,quantity,income_productsale,product_id,sale_id,name,cost,category_id,income_sale,client_id,salesman_id,year,month
0,16921,4.1,14.35,17,00034279,QUESO TELITA CACHAPERO (E),2.70,23,70.15,390,4,2018,11
1,16923,12.0,55.80,254,00034279,BOLOGNA NORMAL 1KG DEL CORRAL X UNIDAD,3.54,19,70.15,390,4,2018,11
2,16922,16.0,0.00,256,00034279,SALCHIPOLLO 450GR DEL CORRAL X UND,0.00,22,70.15,390,4,2018,11
3,16924,4.3,0.00,489,00034279,NONE,0.00,1056,70.15,390,4,2018,11
4,16953,4.0,14.00,17,00034290,QUESO TELITA CACHAPERO (E),2.70,23,235.15,101,4,2018,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71828,82823,1.0,2.28,195,NE010724,REPOSTERIA CHOCOLATE LECHE 250 GRS,1.88,45,2.28,16,1,2021,12
71829,88368,1.0,2.28,195,001001,REPOSTERIA CHOCOLATE LECHE 250 GRS,1.88,45,2.28,16,1,2021,12
71830,82138,2.0,3.69,198,NE010420,REPOSTERIA COBERTURA LECHE 250 GRS,1.23,1064,3.69,16,1,2021,12
71831,82181,8.0,14.77,198,NE010437,REPOSTERIA COBERTURA LECHE 250 GRS,1.23,1064,14.77,16,1,2021,12


In [7]:
# grouping products per month for return stadistics about this.
# This data frame will be used to plot the graphs for see the historic of sales per product month in every year
# This data frame can be use for trimester too

df_groupby_products = df_merged_sales_with_products.groupby(["year", "month", "product_id"])
sales_data = {
    'income': df_groupby_products['income_productsale'].sum(),
    'count': df_groupby_products['id_productsale'].count(),
    'name': df_groupby_products['name'].first(),
}

df_sales_per_months = pd.DataFrame(sales_data)
df_sales_per_months

income  count  \
year month product_id                   
2018 11    9             84.36      3   
           12           114.45      2   
           13           660.02      7   
           17            54.95      4   
           22           296.61      6   
...                        ...    ...   
2022 2     438           77.99      2   
           439         1091.26     13   
           440         2218.03     30   
           443          207.95      3   
           445         2405.48     33   

                                                           name  
year month product_id                                            
2018 11    9                          QUESO BLANCO DE CABRA (E)  
           12                           QUESO BLANCO DE RES (E)  
           13           QUESO BLANCO PASTEURIZADO MIS QUESOS(E)  
           17                        QUESO TELITA CACHAPERO (E)  
           22                      QUESO BLANCO TIPO TELITA (E)  
...                                                         ...  
2022 2     438               SALCHICHA VIENA GRANEL X KG ALIMEX  
           439          SALCHICHA VIENA 800 GRS X UNIDAD ALIMEX  
           440           SALCHICHA VIENA 450GRS X UNIDAD ALIMEX  
           443                   SALCHICHA WIENNER 450GR ALIMEX  
           445         SALCHICHA DE POLLO 450GR ALIMEX EL TUNAL  

[4444 rows x 3 columns]

In [8]:
# Copy df for ML model. The other DF is for chart.
#Then, it just set split point, reset index and
# split the dataframe in two parts, one for train and one for test

df_machine_learning = df_sales_per_months.copy()
df_machine_learning.reset_index(inplace=True)
split_point = 2021
df_machine_learning.drop(df_machine_learning[df_machine_learning['name'] == "NONE"].index, inplace=True)

data_train = df_machine_learning[df_machine_learning['year'] < split_point].copy()
data_valid = df_machine_learning[df_machine_learning['year'] >= split_point].copy()
data_train

,year,month,product_id,income,count,name
0,2018,11,9,84.36,3,QUESO BLANCO DE CABRA (E)
1,2018,11,12,114.45,2,QUESO BLANCO DE RES (E)
2,2018,11,13,660.02,7,QUESO BLANCO PASTEURIZADO MIS QUESOS(E)
3,2018,11,17,54.95,4,QUESO TELITA CACHAPERO (E)
4,2018,11,22,296.61,6,QUESO BLANCO TIPO TELITA (E)
...,...,...,...,...,...,...
2386,2020,12,449,4.34,2,SUERO DE LECHE YARACAL BOLSA 1KG
2387,2020,12,452,808.92,18,MORTADELA TAPARA 1KG ALIMEX (E)
2388,2020,12,456,69.50,1,JAMÏ¿½N COCIDO PIERNA VIGOR
2389,2020,12,458,23.96,1,ESPALDA COCIDA VIGOR


In [9]:
# Add the columns which gonna be predicted.
# It just set the products sales next month using 
# the income of the next products sales in the DF

data_train["sales_next_month"] = data_train.groupby("product_id")["income"].shift(-1)
data_train["sales_next_month_count"] = data_train.groupby("product_id")["count"].shift(-1)
data_valid["sales_next_month"] = data_valid.groupby("product_id")["income"].shift(-1)
data_valid["sales_next_month_count"] = data_valid.groupby("product_id")["count"].shift(-1)

In [10]:
# Drop NaN values for train, otherwise, the code dont work
# Add lag sales next day in both DF train and valid. It is use
# when some info doesnt exist or there has missing values

data_train.dropna(inplace=True)
data_train["lag_sales_next_month"] = data_train.groupby("product_id")["income"].shift(1)
data_train["lag_sales_next_month_count"] = data_train.groupby("product_id")["count"].shift(1)
data_valid["lag_sales_next_month"] = data_valid.groupby("product_id")["income"].shift(1)
data_valid["lag_sales_next_month_count"] = data_valid.groupby("product_id")["count"].shift(1)

In [11]:
# Set diff between sales and sales next day

data_valid["diff_sales_next_month"] = data_valid.groupby("product_id")["income"].diff(1)
data_valid["diff_sales_next_month_count"] = data_valid.groupby("product_id")["count"].diff(1)
data_train["diff_sales_next_month"] = data_train.groupby("product_id")["income"].diff(1)
data_train["diff_sales_next_month_count"] = data_train.groupby("product_id")["count"].diff(1)
data_train

,year,month,product_id,income,count,name,sales_next_month,sales_next_month_count,lag_sales_next_month,lag_sales_next_month_count,diff_sales_next_month,diff_sales_next_month_count
0,2018,11,9,84.36,3,QUESO BLANCO DE CABRA (E),357.72,6.0,NaN,NaN,NaN,NaN
1,2018,11,12,114.45,2,QUESO BLANCO DE RES (E),1104.17,18.0,NaN,NaN,NaN,NaN
2,2018,11,13,660.02,7,QUESO BLANCO PASTEURIZADO MIS QUESOS(E),861.93,6.0,NaN,NaN,NaN,NaN
3,2018,11,17,54.95,4,QUESO TELITA CACHAPERO (E),23.80,2.0,NaN,NaN,NaN,NaN
4,2018,11,22,296.61,6,QUESO BLANCO TIPO TELITA (E),441.15,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2232,2020,11,449,107.06,7,SUERO DE LECHE YARACAL BOLSA 1KG,4.34,2.0,21.71,7.0,85.35,0.0
2233,2020,11,452,1903.79,22,MORTADELA TAPARA 1KG ALIMEX (E),808.92,18.0,691.10,16.0,1212.69,6.0
2234,2020,11,456,453.07,9,JAMÏ¿½N COCIDO PIERNA VIGOR,69.50,1.0,104.90,2.0,348.17,7.0
2235,2020,11,458,310.29,11,ESPALDA COCIDA VIGOR,23.96,1.0,NaN,NaN,NaN,NaN


In [16]:
# Baseline for indicators for precission of the model

data_train_filtered = data_train.copy()
data_train_filtered.reset_index(inplace=True)
data_train_filtered.sort_values(inplace=True, by=["product_id", "year", "month"])


y_pred = data_train_filtered["income"]
y_true = data_train_filtered["sales_next_month"]

print(mape(y_true, y_pred))
wmape(y_true, y_pred)



3.172066005858639


0.5638986031245905

In [13]:
# Set features for ML model, which columns will be use for predict
# Imputer is for transform some data
# Xtr and Ytr are the X and Y values for train the model
# Then the model is just a RandomForestRegressor and with fit it get trained

features = ["income", "count", "lag_sales_next_month", "lag_sales_next_month_count", "diff_sales_next_month", "diff_sales_next_month_count"]
imputer = SimpleImputer()
Xtr_per_month = imputer.fit_transform(data_train_filtered[features])
ytr_per_month = data_train_filtered['sales_next_month']


model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
model.fit(Xtr_per_month, ytr_per_month)

RandomForestRegressor(n_jobs=6, random_state=0)

In [32]:
# Predicting sales for everyh day with the model
# This model just predict the income of the next month, not how many sales

# The final data frame is just for seing true values and predicted values

data_valid_filtered = data_valid.copy()
data_valid_filtered.reset_index(inplace=True)

Xval_per_month = imputer.fit_transform(data_valid_filtered[features])
yval_per_month = data_valid_filtered['sales_next_month']

prediction_per_month = model.predict(Xval_per_month)

df_forest_per_month = pd.DataFrame({"product_id": data_valid_filtered["product_id"],"product": data_valid_filtered["name"],"value": data_valid_filtered["income"],"year": data_valid_filtered["year"], "month": data_valid_filtered["month"] ,"predicted": prediction_per_month, "true": yval_per_month, "diff": data_valid_filtered["diff_sales_next_month"]})
df_forest_per_month[df_forest_per_month["product_id"] == 20]

,product_id,product,value,year,month,predicted,true,diff
260,20,SALCHICHA VIENA 225GR ALIMEX,2865.80,2021,3,5556.5857,601.64,NaN
381,20,SALCHICHA VIENA 225GR ALIMEX,601.64,2021,4,2893.7680,466.36,-2264.16
495,20,SALCHICHA VIENA 225GR ALIMEX,466.36,2021,5,801.1524,768.96,-135.28
609,20,SALCHICHA VIENA 225GR ALIMEX,768.96,2021,6,631.7021,1007.48,302.60
727,20,SALCHICHA VIENA 225GR ALIMEX,1007.48,2021,7,899.2753,779.64,238.52
898,20,SALCHICHA VIENA 225GR ALIMEX,779.64,2021,8,1054.2025,938.06,-227.84
1057,20,SALCHICHA VIENA 225GR ALIMEX,938.06,2021,9,1250.6007,3645.44,158.42
1230,20,SALCHICHA VIENA 225GR ALIMEX,3645.44,2021,10,4515.4540,2940.56,2707.38
1408,20,SALCHICHA VIENA 225GR ALIMEX,2940.56,2021,11,3977.8408,1952.66,-704.88
1599,20,SALCHICHA VIENA 225GR ALIMEX,1952.66,2021,12,2766.4138,2052.34,-987.90


In [29]:
# see the percentage for the error of the model for sales income

print(mape(df_forest_per_month[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month["product_id"] == 20]["predicted"]))
print(wmape(df_forest_per_month[df_forest_per_month["product_id"] == 20]["true"], df_forest_per_month[df_forest_per_month["product_id"] == 20]["predicted"]))

1.637078626680731
0.9916292134831459


In [ ]:
# Baseline for indicators for precission of the model
# This baseline is for how many sales will be

y_pred_count = data_train_filtered["sales_next_month_count"]
y_true_count = data_train_filtered['count']

print(mape(y_true, y_pred))
wmape(y_true, y_pred)

3.172066005858639


0.5638986031245905

In [ ]:
# Set features for ML model, which columns will be use for predict
# Imputer is for transform some data
# Xtr and Ytr are the X and Y values for train the model
# Then the model is just a RandomForestRegressor and with fit it get trained
# Again, same shit, but for sales count

features_count = ["income", "count", "lag_sales_next_month", "lag_sales_next_month_count", "diff_sales_next_month", "diff_sales_next_month_count"]
imputer_count = SimpleImputer()
Xtr_per_month_count = imputer_count.fit_transform(data_train_filtered[features_count])
ytr_per_month_count = data_train_filtered['sales_next_month_count']


model_count = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
model_count.fit(Xtr_per_month_count, ytr_per_month_count)

RandomForestRegressor(n_jobs=6, random_state=0)

In [ ]:
# Predicting sales for everyh day with the model
# This model just predict the income of the next month, not how many sales

# The final data frame is just for seing true values and predicted values


Xval_per_month_count = imputer.fit_transform(data_valid_filtered[features])
yval_per_month_count = data_valid_filtered['sales_next_month_count']

prediction_per_month_count = model_count.predict(Xval_per_month_count)

df_forest_per_month_count = pd.DataFrame({"product": data_valid_filtered["name"],"year": data_valid_filtered["year"], "month": data_valid_filtered["month"] ,"predicted": prediction_per_month_count, "true": yval_per_month_count, "diff_count": data_valid_filtered["diff_sales_next_month_count"]})
df_forest_per_month_count

,product,year,month,predicted,true,diff_count
719,REQUESON DURO CON SAL (E),2021,7,14.200000,NaN,NaN
255,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),2021,3,12.850000,1.0,NaN
376,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),2021,4,3.760000,1.0,-2.0
1593,QUESO PASTEURIZADO PURISIMA EL TUNAL (E),2021,12,4.003333,NaN,0.0
720,CREMOSITO CHEDDAR TINA 220 GRS DERILAC,2021,7,14.165714,6.0,NaN
...,...,...,...,...,...,...
889,ADEREZO DE CELERY NATUGURT,2021,7,13.703333,1.0,NaN
1052,ADEREZO DE CELERY NATUGURT,2021,8,7.280000,NaN,0.0
890,ADEREZO DE CILANTRO NATUGURT,2021,7,14.020000,NaN,NaN
891,ADEREZO DE TOCINETA NATUGURT,2021,7,13.703333,NaN,NaN


In [ ]:
print(mape(yval_per_month_count, prediction_per_month_count))
wmape(yval_per_month_count, prediction_per_month_count)

2.2189529325612827


0.5737227519883611